In [86]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers

from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from create_data_set import clean_data

# from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
# from keras.models import Model

In [109]:
"""
input_layer = Input(shape=(400, 16))
conv1 = Conv1D(filters=32,
               kernel_size=8,
               strides=1,
               activation='relu',
               padding='same')(input_layer)
lstm1 = LSTM(32, return_sequences=True)(conv1)
output_layer = Dense(1, activation='sigmoid')(lstm1)
model = Model(inputs=input_layer, outputs=output_layer)
"""


model = keras.Sequential()

model.add(layers.Conv1D(filters=32,
                        kernel_size=8,
                        strides=1,
                        activation='relu',
                        padding='same'))

model.add(layers.LSTM(32))  # Set return_sequences=False

model.add(layers.Dense(64, kernel_initializer='uniform'))
model.add(layers.Activation('softmax'))

loss_function = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss_function, optimizer='adam')

# model.build()
# model.summary()

SyntaxError: invalid syntax (2661215580.py, line 17)

In [44]:
f_name  = f'C:/Users/adirk/Documents/nano-clustering/hadas_adir_barak_train.csv'
# create_data_set(f_name, 20, 2, False)
col_names = ['signal', 'barcode']
data = pd.read_csv(f_name, index_col=0)
# data.columns = col_names
X = data['signal']
X = X.apply(eval).apply(np.array)
X = X.apply(lambda x: clean_data(x,3000))
Y = data['barcode']
y_signals = list(Y)
y_signals2 = list(set(y_signals))
# Example names vector (should have 20 names)
class_number = [i for i in range(20)]

seed = 211

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

# seprate to test, validation 
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.33, random_state=seed)
print(f"Train size: {x_train.shape[0]}")
print(f"Val size: {x_val.shape[0]}")
print(f"Test size: {x_test.shape[0]}")


Train size: 4172
Val size: 591
Test size: 1197


In [46]:

# # pre-processing and converting labels to integers
# x_train_prep = preprocessing.scale(x_train)
# x_test_prep = preprocessing.scale(x_test)
# x_val_prep = preprocessing.scale(x_val)


y_train_np = np.array([class_number[y_signals2.index(x)] for x in y_train]).astype(int)
y_test_np = np.array([class_number[y_signals2.index(x)] for x in y_test]).astype(int)
y_val_np = np.array([class_number[y_signals2.index(x)] for x in y_val]).astype(int)


# Convert the NumPy arrays to torch tensors
x_train_tensor = torch.tensor(np.stack(x_train))
y_train_tensor = torch.from_numpy(y_train_np).float()

x_val_tensor = torch.tensor(np.stack(x_val))
y_val_tensor = torch.from_numpy(y_val_np).float()
x_test_tensor = torch.tensor(np.stack(x_test))
y_test_tensor = torch.from_numpy(y_test_np).float()
# Create the TensorDataset
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = torch.utils.data.TensorDataset(x_val_tensor, y_val_tensor)
test_dataset = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

In [107]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_train_encoded = label_encoder.fit_transform(y_train_tensor)

y_train_tensor = torch.from_numpy(y_train_encoded).long()
print(np.shape(y_train_tensor))

torch.Size([4172])


In [102]:
print(np.shape(x_train_tensor))
print(np.shape(y_train_tensor))

torch.Size([4172, 3000])
torch.Size([4172])


In [108]:

# Now, you can use the TensorFlow tensors for training
model.fit(x_train_tensor, y_train_encoded_tf, epochs=2000, batch_size=5, verbose=0)

TypeError: Cannot convert the argument `type_value`: torch.int32 to a TensorFlow DType.